In [47]:
%%writefile check_div.py
import math

def div(x: float, y: float) -> float:
    assert y != 0
    assert math.isfinite(x) and math.isfinite(y)
    #print(f"Dividing x={x} y={y}")
    res = x / y
    assert res * y == x
    return res

Overwriting check_div.py


In [48]:
!crosshair check --analysis_kind=asserts check_div.py

In [49]:
!crosshair cover check_div.py

div(0.0, 1.0)
div(0.0, float("inf"))
div(0.0, 0.0)


In [50]:
def div(x: float, y: float) -> float:
    assert y != 0
    res = x / y
    assert res * y == x
    return res


In [58]:
from z3 import Real, Solver, sat

# Declare symbolic variables
x, y = Real('x'), Real('y')

# Preconditions
preconditions = [y != 0]

# Postcondition
res = x / y
postcondition = (res * y == x)

# Ask Z3: can preconditions hold AND postcondition fail?
with Solver() as s:
    
    s.add(preconditions)
    s.add(res * y != x)  # Negation of postcondition

    if s.check() == sat:
        print("Counterexample:", s.model())
    else:
        print("No counterexamples (property holds).")


No counterexamples (property holds).


In [59]:
from z3 import FP, Float32  # Need FP to create floating-point variables

# Use separate variable names to avoid overwriting existing Real variables x, y
x_fp = FP('x_fp', Float32())
y_fp = FP('y_fp', Float32())

# Preconditions for floating point version
preconditions_fp = [y_fp != 0]

# Postcondition (may fail due to rounding)
res_fp = x_fp / y_fp
postcondition_fp = (res_fp * y_fp == x_fp)

# Ask Z3: can preconditions hold AND postcondition fail?
s_fp = Solver()
s_fp.add(preconditions_fp)
s_fp.add(res_fp * y_fp != x_fp)  # Negation of postcondition

if s_fp.check() == sat:
    print("Counterexample (floating-point):", s_fp.model())
else:
    print("No counterexamples (floating-point property holds).")


Counterexample (floating-point): [y_fp = NaN, x_fp = -1.4716269969940185546875*(2**-96)]
